# Machine Learning en la extracción de minerales

### El objetivo de este proyecto, es crear un modelo de machine learning que deba predecir la cantidad de oro extraido en minerales de oro.

## ¿Como se extrae el oro del mineral?

El mineral extraído se somete a un tratamiento primario para obtener la mezcla de mineral, o alimentación rougher, que es la materia prima utilizada para la flotación (también conocida como proceso rougher). Después de la flotación, el material se somete al proceso de purificación en dos etapas.
Estas 2 etapas son:
### 1) Flotación
La mezcla de mineral de oro se introduce en las plantas de flotación para obtener un concentrado de oro rougher y colas rougher (es decir, residuos del producto con una baja concentración de metales valiosos).

La estabilidad de este proceso se ve afectada por la volatilidad y el estado físico-químico desfavorable de la pulpa de flotación (una mezcla de partículas sólidas y líquido).
### 2) Purificación
El concentrado rougher se somete a dos etapas de purificación. Tras esto, tenemos el concentrado final y las nuevas colas.

## Descripción de los datos

1) Proceso tecnológico

- Rougher feed: materia prima
- Rougher additions (o adiciones de reactivos): reactivos de flotación: xantato, sulfato, depresante
- Xantato: promotor o activador de la flotación
- Sulfato: sulfuro de sodio para este proceso en particular
- Depresante: silicato de sodio
- Rougher process: flotación
- Rougher tails: residuos del producto
- Float banks: instalación de flotación
- Cleaner process: purificación
- Rougher Au: concentrado de oro rougher
- Final Au: concentrado de oro final

2) Parámetros de las etapas
- air amount: volumen de aire
- fluid levels
- feed size: tamaño de las partículas de la alimentación
- feed rate

## Denominación de las características
Así es como se denominan las características:

[stage].[parameter_type].[parameter_name]

Ejemplo: rougher.input.feed_ag

Valores posibles para [stage]:

- rougher: flotación
- primary_cleaner: purificación primaria
- secondary_cleaner: purificación secundaria
- final: características finales
- Valores posibles para [parameter_type]:

- input: parámetros de la materia prima
- output: parámetros del producto
- state: parámetros que caracterizan el estado actual de la etapa
- calculation: características de cálculo

## Cálculo del la recuperación 
$$
\text{Recuperación} = \frac{C \cdot (F - T)}{F \cdot (C - T)} \times 100\%
$$

### Donde:

C
- Para saber la recuperación del concentrado rougher → la proporción de oro en el concentrado justo después de la flotación o
- Para saber la recuperación del concentrado final → la proporción de oro después de la purificación.

F
- Para saber la recuperación del concentrado rougher → la proporción de oro en la alimentación antes de la flotación
- Para saber la recuperación del concentrado final → la proporción de oro en el concentrado justo después de la flotación.
  
T
- Para saber la recuperación del concentrado rougher → la proporción de oro en las colas rougher justo después de la flotación.
- Para saber la recuperación del concentrado final → la proporción de oro después de la purificación.

Para predecir el coeficiente, hay que encontrar la proporción de oro en el concentrado y en las colas.

## Métricas de evaluación
Para resolver el problema, necesitaremos una nueva métrica. Se llama sMAPE, o error medio absoluto porcentual simétrico.

Es similar al MAE, pero se expresa en valores relativos en lugar de absolutos. ¿Por qué es simétrico? Porque tiene en cuenta la escala tanto del objetivo como de la predicción.

Así es como se calcula el sMAPE:

$$
sMAPE = \frac{1}{N} \sum_{i=1}^{N} 
\left( \frac{ \lvert y_i - \hat{y}_i \rvert }{ \frac{ \lvert y_i \rvert + \lvert \hat{y}_i \rvert }{2} } \right) \times 100\%
$$


### Donde:
- yi: Valor objetivo
- yí: Valor de la predicción
- N: Numero de observaciones de la muestra
- Σ: Suma de todas las observaciones de la muestra (tomamos el valor de i=1 como el valor inicial de 1 a N)

### Necesitamos predecir dos valores:

1) La recuperación del concentrado rougher rougher.output.recovery.
2) La recuperación final del concentrado final.output.recovery.

### La métrica final incluye los dos valores:
$$
sMAPE_{\text{final}} = 25\% \cdot sMAPE_{\text{rougher}} \;+\; 75\% \cdot sMAPE_{\text{final}}
$$


In [1]:
#Carga de las librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [17]:
#Carga de los datasets (Cada dataset se le asigna una variable diferente para mayor estabilidad y manejo de datos)
train = pd.read_csv('gold_recovery_train.csv', parse_dates=['date'], index_col='date')
test = pd.read_csv('gold_recovery_test.csv', parse_dates=['date'], index_col='date')
full = pd.read_csv('gold_recovery_full.csv', parse_dates=['date'], index_col='date')
# Verificar dimensiones y columnas
print("Train:", train.shape)
print("Test:", test.shape)
print("Full:", full.shape)

print("\nColumnas en train:")
print(train.columns.tolist())
print("\nColumnas en test:")
print(test.columns.tolist())
print("\nColumnas en full:")
print(full.columns.tolist())


Train: (16860, 86)
Test: (5856, 52)
Full: (22716, 86)

Columnas en train:
['final.output.concentrate_ag', 'final.output.concentrate_pb', 'final.output.concentrate_sol', 'final.output.concentrate_au', 'final.output.recovery', 'final.output.tail_ag', 'final.output.tail_pb', 'final.output.tail_sol', 'final.output.tail_au', 'primary_cleaner.input.sulfate', 'primary_cleaner.input.depressant', 'primary_cleaner.input.feed_size', 'primary_cleaner.input.xanthate', 'primary_cleaner.output.concentrate_ag', 'primary_cleaner.output.concentrate_pb', 'primary_cleaner.output.concentrate_sol', 'primary_cleaner.output.concentrate_au', 'primary_cleaner.output.tail_ag', 'primary_cleaner.output.tail_pb', 'primary_cleaner.output.tail_sol', 'primary_cleaner.output.tail_au', 'primary_cleaner.state.floatbank8_a_air', 'primary_cleaner.state.floatbank8_a_level', 'primary_cleaner.state.floatbank8_b_air', 'primary_cleaner.state.floatbank8_b_level', 'primary_cleaner.state.floatbank8_c_air', 'primary_cleaner.state.f

En este caso, antes de trabajar con los datos, debemos cargar y verificar las dimensiones de los datasets. Este paso inicial es crucial para entender la estructura y el alcance de los datos, lo que permitirá planificar adecuadamente los siguientes pasos en el análisis.

In [18]:
#Verificacion de valores nulos y duplicados 
def resumen_nulos(df, nombre):
    nulos = df.isnull().sum()
    porcentaje = (nulos / len(df)) * 100
    resumen = pd.DataFrame({'nulos': nulos, '%': porcentaje})
    print(f'\nValores nulos en {nombre}:')
    return resumen[resumen['nulos'] > 0].sort_values(by='%', ascending=False)
def resumen_duplicados(df, nombre):
    duplicados = df.duplicated().sum()
    print(f'{nombre} tiene {duplicados} filas duplicadas.')
    return df.drop_duplicates()
# Verificar nulos y duplicados
nulos_train = resumen_nulos(train, 'train')
nulos_test = resumen_nulos(test, 'test')
nulos_full = resumen_nulos(full, 'full')
train_clean = resumen_duplicados(train, 'train')
test_clean = resumen_duplicados(test, 'test')
full_clean = resumen_duplicados(full, 'full')
# Eliminar nulos en train y full
train_clean = train_clean.dropna()
full_clean = full_clean.dropna()
# Imputar nulos en test
test_clean = test_clean.fillna(method='ffill').fillna(method='bfill')


Valores nulos en train:

Valores nulos en test:

Valores nulos en full:
train tiene 16 filas duplicadas.
test tiene 6 filas duplicadas.
full tiene 22 filas duplicadas.


C:\Users\EndUser\AppData\Local\Temp\ipykernel_19152\3430195324.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_clean = test_clean.fillna(method='ffill').fillna(method='bfill')


### Justificación
El manejo de valores nulos y duplicados constituye una etapa crítica en la preparación de datos, ya que afecta directamente la validez de los modelos predictivos. En este caso, se optó por eliminar los duplicados en todos los conjuntos para evitar sesgos y redundancias que pudieran distorsionar los resultados.
Respecto a los valores nulos, la estrategia fue diferenciada:
- En los conjuntos train y full, se eliminaron las observaciones incompletas con el fin de preservar la integridad y confiabilidad del aprendizaje del modelo, evitando introducir ruido mediante imputaciones artificiales.
- En el conjunto test, se aplicó imputación mediante propagación hacia adelante y hacia atrás, priorizando la representatividad y completitud del conjunto de evaluación, de modo que el modelo pudiera ser probado en todas las instancias disponibles.
En síntesis, la decisión combina criterios de robustez en el entrenamiento y amplitud en la evaluación, garantizando tanto la calidad del aprendizaje como la validez de la medición del desempeño del modelo.


In [19]:
#Verificación del cálculo de recuperación en train test
# Columnas específicas para oro
cols_recovery_au = [
    'rougher.input.feed_au',
    'rougher.output.tail_au',
    'rougher.output.concentrate_au',
    'rougher.output.recovery'
]
# Verificar que todas las columnas existen
for col in cols_recovery_au:
    if col not in train_clean.columns:
        print(f"Columna faltante: {col}")
# Filtrar filas completas
recovery_data = train_clean[cols_recovery_au].dropna()
# Asignar variables
F = recovery_data['rougher.input.feed_au']
T = recovery_data['rougher.output.tail_au']
C = recovery_data['rougher.output.concentrate_au']
R_real = recovery_data['rougher.output.recovery']

# Cálculo de recuperación
R_calc = (C * (F - T)) / (F * (C - T)) * 100

# Error Absoluto Medio
eam = abs(R_real - R_calc).mean()
print(f'EAM para recuperación de oro (au): {eam:.2f}')

EAM para recuperación de oro (au): 0.00


La verificación del cálculo de recuperación de oro es precisa, y el hecho de que el error absoluto medio sea cero indica que el cálculo es correcto. Asegurarnos de que las columnas necesarias estén presentes es un paso importante para garantizar la validez de tus cálculos.

In [20]:
#Verificación del cálculo de recuperación en test clean
print("Columnas disponibles en test_clean:")
print(test_clean.columns.tolist())

missing_cols = set(train_clean.columns) - set(test_clean.columns)

print("\nColumnas ausentes en test_clean respecto a train_clean:")
for col in sorted(missing_cols):
    print(f"- {col}")
# Eliminar columnas objetivo de train_clean
features_train = train_clean.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1)

# Alinear columnas entre train y test
common_cols = features_train.columns.intersection(test_clean.columns)

# Filtrar ambos datasets con las columnas comunes
features_train = features_train[common_cols]
features_test = test_clean[common_cols]
print(f"\nColumnas comunes para modelado: {len(common_cols)}")

Columnas disponibles en test_clean:
['primary_cleaner.input.sulfate', 'primary_cleaner.input.depressant', 'primary_cleaner.input.feed_size', 'primary_cleaner.input.xanthate', 'primary_cleaner.state.floatbank8_a_air', 'primary_cleaner.state.floatbank8_a_level', 'primary_cleaner.state.floatbank8_b_air', 'primary_cleaner.state.floatbank8_b_level', 'primary_cleaner.state.floatbank8_c_air', 'primary_cleaner.state.floatbank8_c_level', 'primary_cleaner.state.floatbank8_d_air', 'primary_cleaner.state.floatbank8_d_level', 'rougher.input.feed_ag', 'rougher.input.feed_pb', 'rougher.input.feed_rate', 'rougher.input.feed_size', 'rougher.input.feed_sol', 'rougher.input.feed_au', 'rougher.input.floatbank10_sulfate', 'rougher.input.floatbank10_xanthate', 'rougher.input.floatbank11_sulfate', 'rougher.input.floatbank11_xanthate', 'rougher.state.floatbank10_a_air', 'rougher.state.floatbank10_a_level', 'rougher.state.floatbank10_b_air', 'rougher.state.floatbank10_b_level', 'rougher.state.floatbank10_c_air

In [21]:
#Alineación de las columnas entre train_clean y test_clean
# Eliminar columnas objetivo del conjunto de entrenamiento
features_train = train_clean.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1)
# Identificar columnas comunes
common_cols = features_train.columns.intersection(test_clean.columns)
# Filtrar ambos datasets
features_train = features_train[common_cols]
features_test = test_clean[common_cols]
print(f"\nColumnas alineadas para modelado: {len(common_cols)}")


Columnas alineadas para modelado: 52


In [22]:
#Separar variables objetivo
target_rougher = train_clean['rougher.output.recovery']
target_final = train_clean['final.output.recovery']

In [23]:
#Definir la métrica de sMAPE
def smape(y_true, y_pred):
    denominator = (abs(y_true) + abs(y_pred)) / 2
    return (abs(y_true - y_pred) / denominator).mean() * 100
def final_smape(y_true_rougher, y_pred_rougher, y_true_final, y_pred_final):
    smape_r = smape(y_true_rougher, y_pred_rougher)
    smape_f = smape(y_true_final, y_pred_final)
    return 0.25 * smape_r + 0.75 * smape_f
# Crear scorer compatible con cross_val_score
smape_scorer = make_scorer(smape, greater_is_better=False)

In [24]:
#Separación características y objetivo
X = features_train.copy()
X_test = features_test.copy()
y_rougher = train_clean['rougher.output.recovery']
y_final = train_clean['final.output.recovery']

In [25]:
#Validación con Random Forest
# Modelo para rougher
model_rougher = RandomForestRegressor(n_estimators=100, random_state=42)
scores_rougher = cross_val_score(model_rougher, X, y_rougher, cv=5, scoring=smape_scorer)
# Modelo para final
model_final = RandomForestRegressor(n_estimators=100, random_state=42)
scores_final = cross_val_score(model_final, X, y_final, cv=5, scoring=smape_scorer)

In [26]:
#Mostrar los resultados
# Convertir a valores positivos (porque greater_is_better=False)
mean_rougher = -scores_rougher.mean()
mean_final = -scores_final.mean()
# Métrica combinada
smape_final_score = 0.25 * mean_rougher + 0.75 * mean_final
# Imprimir resultados
print("sMAPE Rougher (promedio):", round(mean_rougher, 2))
print("sMAPE Final (promedio):", round(mean_final, 2))
print("sMAPE Combinado (25% Rougher + 75% Final):", round(smape_final_score, 2))

sMAPE Rougher (promedio): 13.01
sMAPE Final (promedio): 10.43
sMAPE Combinado (25% Rougher + 75% Final): 11.07


1) El modelo base (Random Forest sin tuning) ya ofrece un rendimiento competitivo.
2) El rougher tiene más variabilidad, lo que podría mejorar con ingeniería de características o modelos más sensibles a interacciones.

In [27]:
#Modelo de regresión lineal
lr_rougher = LinearRegression()
scores_lr_rougher = cross_val_score(lr_rougher, X, y_rougher, cv=5, scoring=smape_scorer)
# Modelo para final
lr_final = LinearRegression()
scores_lr_final = cross_val_score(lr_final, X, y_final, cv=5, scoring=smape_scorer)
# Convertir a valores positivos
mean_lr_rougher = -scores_lr_rougher.mean()
mean_lr_final = -scores_lr_final.mean()
smape_lr_combined = 0.25 * mean_lr_rougher + 0.75 * mean_lr_final

In [28]:
#Comparación de los 2 modelos
print("Linear Regression:")
print("   sMAPE Rougher:", round(mean_lr_rougher, 2))
print("   sMAPE Final:", round(mean_lr_final, 2))
print("   sMAPE Combinado:", round(smape_lr_combined, 2))

print("\nRandom Forest (previo):")
print("   sMAPE Rougher:", round(mean_rougher, 2))
print("   sMAPE Final:", round(mean_final, 2))
print("   sMAPE Combinado:", round(smape_final_score, 2))

Linear Regression:
   sMAPE Rougher: 11.93
   sMAPE Final: 9.75
   sMAPE Combinado: 10.29

Random Forest (previo):
   sMAPE Rougher: 13.01
   sMAPE Final: 10.43
   sMAPE Combinado: 11.07


### Conclusiones
1. Linear Regression es mejor (10.29% vs 11.07%)
- Diferencia de 0.78 puntos porcentuales
- Es más simple y predice mejor

2. Ambos modelos predicen mejor el proceso "Final"
- Linear: 9.75% vs Random Forest: 10.43%
- El proceso final parece más predecible

3. El proceso "Rougher" es más difícil de predecir
- Ambos modelos tienen errores más altos aquí